# Phase 1: Dirty Data Diagnostics 🩺

**Goal**: Diagnose data quality issues (gaps, noise, format errors) in the Raw Bronze Layer (S3/MinIO) to define the "Cleaning Rules" for Phase 2.

**Dataset Context**:
*   **Source**: Nature Scientific Data (2022) - Pharma Manufacturing.
*   **Components**:
    1.  `Laboratory.csv`: Quality Targets (CQAs) & Metadata.
    2.  `Process.csv`: Aggregated features (1 row per batch).
    3.  `Process/*.csv`: **Raw Time Series** (Sensor data @ 10s frequency).

## 1. Setup & Ingestion

In [1]:
# Import Cloud-Agnostic Config
import sys
import os
sys.path.append(os.path.abspath('../src'))

from config import get_spark_session, get_data_path, get_boto3_client
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Initialize Spark
spark = get_spark_session("DirtyEDA_Pharma")

bucket = os.getenv("BUCKET_BRONZE", "bronze")
print(f"🌍 Reading from Bucket: {bucket}")

🔧 Configuring specific S3 endpoint for MinIO: http://minio:9000


:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8ebfd4e2-aca4-497e-b108-9e7098ea2ed7;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central


:: resolution report :: resolve 202ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-8ebfd4e2-aca4-497e-b108-9e7098ea2ed7
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/5ms)
26/01/21 16:18:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes whe

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


26/01/21 16:18:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


🌍 Reading from Bucket: bronze


## 2. Metadata & Aggregated Data (Batch Level)
First, we check the metadata files (`Laboratory`, `Process` aggregated).

In [2]:
# Define Paths
path_lab = f"s3a://{bucket}/Laboratory.csv"
path_proc_agg = f"s3a://{bucket}/Process.csv"
path_norm = f"s3a://{bucket}/Normalization.csv"

# Load Data (Explicit Semicolon Delimiter for CSVs from these authors)
df_lab = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "true").csv(path_lab)
df_proc_agg = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "true").csv(path_proc_agg)

print(f"📊 Laboratory Count: {df_lab.count()} rows (Batches)")
print(f"📊 Process Aggregated Count: {df_proc_agg.count()} rows (Batches)")

# Sample Check
df_lab.select("batch", "code", "dissolution_av").show(3)

26/01/21 16:18:16 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


📊 Laboratory Count: 1005 rows (Batches)
📊 Process Aggregated Count: 1005 rows (Batches)


+-----+----+--------------+
|batch|code|dissolution_av|
+-----+----+--------------+
|    1|  25|         93.83|
|    2|  25|         99.67|
|    3|  25|         97.33|
+-----+----+--------------+
only showing top 3 rows



## 3. Raw Time Series Analysis (`Process/*.csv`)
This is the **High-Frequency Sensor Data** (100s of files). We load them all into a single DataFrame.

*Note: In production, we assume keys are partitioned like `Process/file.csv`.*

In [3]:
path_ts = f"s3a://{bucket}/Process/*.csv"

# Load ALL Time Series files
# Note: The user confirmed TS files also use semicolon delimiter.

try:
    df_ts = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "true").csv(path_ts)
except Exception as e:
    print(f"⚠️ Error loading TS: {e}")

print(f"📈 Time Series Total Rows: {df_ts.count()}")
df_ts.printSchema()

📈 Time Series Total Rows: 4720208
root
 |-- timestamp: string (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- batch: integer (nullable = true)
 |-- code: integer (nullable = true)
 |-- tbl_speed: double (nullable = true)
 |-- fom: double (nullable = true)
 |-- main_comp: double (nullable = true)
 |-- tbl_fill: double (nullable = true)
 |-- SREL: double (nullable = true)
 |-- pre_comp: double (nullable = true)
 |-- produced: integer (nullable = true)
 |-- waste: integer (nullable = true)
 |-- cyl_main: double (nullable = true)
 |-- cyl_pre: double (nullable = true)
 |-- stiffness: integer (nullable = true)
 |-- ejection: integer (nullable = true)



## 4. Gap Analysis (Temporal Continuity)
We need to find missing data points in the Time Series.
*   **Expected Frequency**: 10 seconds.
*   **Risk**: If Delta > 10s, we have a gap.

In [4]:
# Ensure 'time_stamp' column exists (name varies in raw data: 'Time', 'timestamp', 'time_stamp')
# We'll search for it.
ts_col = [c for c in df_ts.columns if 'time' in c.lower()]
if ts_col:
    target_ts_col = ts_col[0]
    print(f"⏱️ Found Timestamp Column: {target_ts_col}")
    
    # Cast to timestamp
    df_ts_clean = df_ts.withColumn("ts_parsed", F.to_timestamp(F.col(target_ts_col)))
    
    # Window for Lag
    # Note: Raw TS might not have 'batch' column inside. If not, we rely on file input_file_name() or just assume continuity per file.
    # For Dirty EDA, we analyze global gaps.
    w = Window.orderBy("ts_parsed")
    
    df_gaps = df_ts_clean.withColumn("prev_ts", F.lag("ts_parsed").over(w)) \
        .withColumn("delta_sec", F.col("ts_parsed").cast("long") - F.col("prev_ts").cast("long"))
    
    print("⚠️ Top 5 Gaps (Seconds):")
    df_gaps.filter("delta_sec > 11").orderBy(F.desc("delta_sec")).show(5)
else:
    print("❌ No Timestamp column found in Time Series schema!")

⏱️ Found Timestamp Column: timestamp
⚠️ Top 5 Gaps (Seconds):


26/01/21 16:18:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 16:18:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 16:18:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


26/01/21 16:18:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
26/01/21 16:18:37 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------------------+--------+-----+----+---------+---+---------+--------+----+--------+--------+-----+--------+-------+---------+--------+-------------------+-------------------+---------+
|          timestamp|campaign|batch|code|tbl_speed|fom|main_comp|tbl_fill|SREL|pre_comp|produced|waste|cyl_main|cyl_pre|stiffness|ejection|          ts_parsed|            prev_ts|delta_sec|
+-------------------+--------+-----+----+---------+---+---------+--------+----+--------+--------+-----+--------+-------+---------+--------+-------------------+-------------------+---------+
|2018-11-13 06:16:33|       3|    1|  25|      0.0|0.0|      0.8|    5.68| 0.0|     0.0|       0|    0|    1.79|    6.0|       50|     107|2018-11-13 06:16:33|2018-06-12 22:10:55| 13248338|
|2020-04-28 21:40:41|     131|  622|  21|      0.0|0.0|      4.6|    6.01| 6.0|     0.0|       0|    0|     1.9|    8.0|       41|     177|2020-04-28 21:40:41|2020-01-30 17:48:56|  7703505|
|2020-08-06 02:42:01|     135|  667|  21|      0.0

## 5. Noise & Outlier Profiling
Focus on Critical Process Parameters (CPPs).
*   **Target**: `main_comp` (Compression Force in kN).
*   **Rules**: No Negatives. Check 5-Sigma.

In [5]:
# Find Main Compression Column
# Likely 'main_comp' or similar
comp_cols = [c for c in df_ts.columns if 'main' in c.lower() and 'comp' in c.lower()]

if comp_cols:
    target_col = comp_cols[0]
    print(f"🎯 Analyzing Sensor: {target_col}")
    
    # 1. Cast to Double (Crucial explicitly)
    df_ts = df_ts.withColumn(target_col, F.col(target_col).cast("double"))
    
    # 2. Negative Values (Physical Impossibility)
    neg_count = df_ts.filter(F.col(target_col) < 0).count()
    print(f"   ❌ Negative Values: {neg_count}")
    
    # 3. Spikes (5 Sigma)
    stats = df_ts.select(F.mean(target_col).alias("mu"), F.stddev(target_col).alias("sigma")).collect()[0]
    
    if stats['mu'] is not None and stats['sigma'] is not None:
        upper = stats['mu'] + 5 * stats['sigma']
        lower = stats['mu'] - 5 * stats['sigma']
        
        outliers = df_ts.filter((F.col(target_col) > upper) | (F.col(target_col) < lower)).count()
        print(f"   📉 Outliers (> 5σ): {outliers} / {df_ts.count()}")
        print(f"   📊 Mean: {stats['mu']:.2f}, StdDev: {stats['sigma']:.2f}")
    else:
        print("   ⚠️ Stats is None. Checking data quality...")
        df_ts.select(target_col).show(5)
else:
    print("❌ Main Compression column not found!")

🎯 Analyzing Sensor: main_comp


   ❌ Negative Values: 0


   📉 Outliers (> 5σ): 2499 / 4720208
   📊 Mean: 6.21, StdDev: 2.13


## 6. The Cleaning Manifesto 📜

**Data-Driven Rules for Phase 2 (ETL Pipeline)**:

| ID | Data Quality Issue | Findings in Bronze | Action Rule | Severity |
|----|--------------------|--------------------|-------------|----------|
| **R1** | **Negative Pressures** | **0 found**. Data is physically consistent. | **Verify & Monitor**. (Do not implement aggressive drops yet). | Low |
| **R2** | **Temporal Gaps** | Huge gaps found (> 10M sec) between campaigns. | **Split Sequences**. Use `campaign` ID as the primary partition key. | **Critical** |
| **R3** | **Sensor Outliers** | ~2,500 points > 5$\sigma$ (0.05% of data). | **Cap/Clip** at $\mu \pm 5\sigma$ or use Robust Scaler. | Medium |
| **R4** | **Schema** | Delimiters vary (`;` vs `,`). | **Enforce `;`** for all Source-of-Truth reading. | High |
| **R5** | **Normalization** | Variable scales differ (kN vs RPM). | **StandardScaler** (Z-Score) required for Deep Learning. | High |